In [1]:

import json
import re
from datetime import datetime
from collections import defaultdict
import uuid
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive

print("Note-taking app initialized successfully!")




Note-taking app initialized successfully!


In [2]:
class NoteManager:
    def __init__(self):
        self.notes = {}
        self.tags = defaultdict(set)  # tag -> set of note_ids
        self.categories = defaultdict(set)  # category -> set of note_ids

    def create_note(self, title, content, category="General", tags=None):
        """Create a new note with title, content, category and tags"""
        note_id = str(uuid.uuid4())[:8]

        # Process tags
        if tags is None:
            tags = []
        elif isinstance(tags, str):
            tags = [tag.strip() for tag in tags.split(',') if tag.strip()]

        note = {
            'id': note_id,
            'title': title.strip(),
            'content': content.strip(),
            'category': category.strip(),
            'tags': tags,
            'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'updated_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'word_count': len(content.split())
        }

        self.notes[note_id] = note

        # Update indices
        self.categories[category].add(note_id)
        for tag in tags:
            self.tags[tag].add(note_id)

        return note_id

    def update_note(self, note_id, title=None, content=None, category=None, tags=None):
        """Update an existing note"""
        if note_id not in self.notes:
            return False

        note = self.notes[note_id]

        # Remove from old indices
        self.categories[note['category']].discard(note_id)
        for tag in note['tags']:
            self.tags[tag].discard(note_id)

        # Update fields
        if title is not None:
            note['title'] = title.strip()
        if content is not None:
            note['content'] = content.strip()
            note['word_count'] = len(content.split())
        if category is not None:
            note['category'] = category.strip()
        if tags is not None:
            if isinstance(tags, str):
                tags = [tag.strip() for tag in tags.split(',') if tag.strip()]
            note['tags'] = tags

        note['updated_at'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Update indices
        self.categories[note['category']].add(note_id)
        for tag in note['tags']:
            self.tags[tag].add(note_id)

        return True

    def delete_note(self, note_id):
        """Delete a note by ID"""
        if note_id not in self.notes:
            return False

        note = self.notes[note_id]

        # Remove from indices
        self.categories[note['category']].discard(note_id)
        for tag in note['tags']:
            self.tags[tag].discard(note_id)

        del self.notes[note_id]
        return True

    def search_notes(self, query, search_in=None):
        """Search notes by content, title, tags, or category"""
        if not query.strip():
            return list(self.notes.values())

        query_lower = query.lower()
        results = []

        for note in self.notes.values():
            match = False

            if search_in is None or 'title' in search_in:
                if query_lower in note['title'].lower():
                    match = True

            if search_in is None or 'content' in search_in:
                if query_lower in note['content'].lower():
                    match = True

            if search_in is None or 'tags' in search_in:
                if any(query_lower in tag.lower() for tag in note['tags']):
                    match = True

            if search_in is None or 'category' in search_in:
                if query_lower in note['category'].lower():
                    match = True

            if match:
                results.append(note)

        # Sort by relevance (updated_at desc)
        return sorted(results, key=lambda x: x['updated_at'], reverse=True)

    def get_all_notes(self, sort_by='updated_at'):
        """Get all notes sorted by specified field"""
        notes_list = list(self.notes.values())
        return sorted(notes_list, key=lambda x: x[sort_by], reverse=True)

    def get_categories(self):
        """Get all categories with note counts"""
        return {cat: len(note_ids) for cat, note_ids in self.categories.items() if note_ids}

    def get_tags(self):
        """Get all tags with note counts"""
        return {tag: len(note_ids) for tag, note_ids in self.tags.items() if note_ids}

    def get_note_by_id(self, note_id):
        """Get a specific note by ID"""
        return self.notes.get(note_id)

    def get_stats(self):
        """Get overall statistics"""
        if not self.notes:
            return {
                'total_notes': 0,
                'total_words': 0,
                'categories': 0,
                'tags': 0
            }

        total_words = sum(note['word_count'] for note in self.notes.values())
        return {
            'total_notes': len(self.notes),
            'total_words': total_words,
            'categories': len([cat for cat, notes in self.categories.items() if notes]),
            'tags': len([tag for tag, notes in self.tags.items() if notes])
        }

# Initialize note manager
note_manager = NoteManager()
print("Note manager initialized!")

Note manager initialized!


In [3]:
def create_note_interface():
    """Interactive interface for creating notes"""
    print("CREATE NEW NOTE")
    print("=" * 50)

    title = input("Note Title: ").strip()
    if not title:
        print("Title is required!")
        return None

    print("\nEnter note content (press Enter twice to finish):")
    content_lines = []
    empty_lines = 0

    while empty_lines < 2:
        line = input()
        if line.strip() == "":
            empty_lines += 1
        else:
            empty_lines = 0
        content_lines.append(line)

    # Remove the extra empty lines at the end
    while content_lines and content_lines[-1].strip() == "":
        content_lines.pop()

    content = "\n".join(content_lines)

    if not content.strip():
        print("Content cannot be empty!")
        return None

    category = input("\nCategory (default: General): ").strip()
    if not category:
        category = "General"

    tags_input = input("Tags (comma-separated): ").strip()

    note_id = note_manager.create_note(title, content, category, tags_input)

    print(f"\nNote created successfully!")
    print(f"Note ID: {note_id}")
    print(f"Title: {title}")
    print(f"Category: {category}")
    print(f"Word count: {len(content.split())}")

    return note_id

# Create your first note
create_note_interface()

CREATE NEW NOTE
Note Title: Khuzi

Enter note content (press Enter twice to finish):
Khuzi WUzi loves yiu
jhuzi



Category (default: General): my
Tags (comma-separated): mine

Note created successfully!
Note ID: 0034ecda
Title: Khuzi
Category: my
Word count: 5


'0034ecda'

In [4]:

def search_notes_interface():
    """Interactive search interface"""
    print("SEARCH NOTES")
    print("=" * 50)

    query = input("Enter search query: ").strip()

    print("\nSearch in:")
    print("1. All fields (default)")
    print("2. Title only")
    print("3. Content only")
    print("4. Tags only")
    print("5. Category only")

    choice = input("Select option (1-5): ").strip()

    search_in = None
    if choice == "2":
        search_in = ['title']
    elif choice == "3":
        search_in = ['content']
    elif choice == "4":
        search_in = ['tags']
    elif choice == "5":
        search_in = ['category']

    results = note_manager.search_notes(query, search_in)

    print(f"\nFound {len(results)} notes:")
    print("=" * 50)

    if not results:
        print("No notes found matching your search.")
        return

    for i, note in enumerate(results, 1):
        print(f"{i}. {note['title']}")
        print(f"   ID: {note['id']}")
        print(f"   Category: {note['category']}")
        print(f"   Tags: {', '.join(note['tags']) if note['tags'] else 'None'}")
        print(f"   Updated: {note['updated_at']}")
        print(f"   Preview: {note['content'][:100]}...")
        print()

    return results

def display_note(note_id):
    """Display a complete note"""
    note = note_manager.get_note_by_id(note_id)
    if not note:
        print(f"Note with ID {note_id} not found!")
        return

    print("NOTE DETAILS")
    print("=" * 50)
    print(f"Title: {note['title']}")
    print(f"ID: {note['id']}")
    print(f"Category: {note['category']}")
    print(f"Tags: {', '.join(note['tags']) if note['tags'] else 'None'}")
    print(f"Created: {note['created_at']}")
    print(f"Updated: {note['updated_at']}")
    print(f"Word count: {note['word_count']}")
    print()
    print("CONTENT:")
    print("-" * 20)
    print(note['content'])
    print()

# Search your notes
search_results = search_notes_interface()


SEARCH NOTES
Enter search query: Kh

Search in:
1. All fields (default)
2. Title only
3. Content only
4. Tags only
5. Category only
Select option (1-5): 1

Found 1 notes:
1. Khuzi
   ID: 0034ecda
   Category: my
   Tags: mine
   Updated: 2025-08-10 22:41:02
   Preview: Khuzi WUzi loves yiu
jhuzi...



In [5]:

def list_all_notes():
    """List all notes with basic info"""
    notes = note_manager.get_all_notes()

    print("ALL NOTES")
    print("=" * 50)

    if not notes:
        print("No notes found.")
        return

    for i, note in enumerate(notes, 1):
        print(f"{i}. {note['title']}")
        print(f"   ID: {note['id']}")
        print(f"   Category: {note['category']}")
        print(f"   Tags: {', '.join(note['tags']) if note['tags'] else 'None'}")
        print(f"   Updated: {note['updated_at']}")
        print(f"   Words: {note['word_count']}")
        print()

    return notes

def edit_note_interface():
    """Interactive interface for editing notes"""
    print("EDIT NOTE")
    print("=" * 50)

    note_id = input("Enter note ID to edit: ").strip()
    note = note_manager.get_note_by_id(note_id)

    if not note:
        print(f"Note with ID {note_id} not found!")
        return False

    print(f"\nCurrent note: {note['title']}")
    print("Leave field empty to keep current value")
    print()

    new_title = input(f"New title (current: {note['title']}): ").strip()

    print(f"\nCurrent content preview: {note['content'][:100]}...")
    update_content = input("Update content? (y/n): ").lower() == 'y'
    new_content = None

    if update_content:
        print("Enter new content (press Enter twice to finish):")
        content_lines = []
        empty_lines = 0

        while empty_lines < 2:
            line = input()
            if line.strip() == "":
                empty_lines += 1
            else:
                empty_lines = 0
            content_lines.append(line)

        while content_lines and content_lines[-1].strip() == "":
            content_lines.pop()

        new_content = "\n".join(content_lines)

    new_category = input(f"New category (current: {note['category']}): ").strip()

    current_tags = ', '.join(note['tags']) if note['tags'] else 'None'
    new_tags = input(f"New tags (current: {current_tags}): ").strip()

    # Update note
    success = note_manager.update_note(
        note_id,
        title=new_title if new_title else None,
        content=new_content if new_content else None,
        category=new_category if new_category else None,
        tags=new_tags if new_tags else None
    )

    if success:
        print("Note updated successfully!")
        return True
    else:
        print("Failed to update note!")
        return False

def delete_note_interface():
    """Interactive interface for deleting notes"""
    print("DELETE NOTE")
    print("=" * 50)

    note_id = input("Enter note ID to delete: ").strip()
    note = note_manager.get_note_by_id(note_id)

    if not note:
        print(f"Note with ID {note_id} not found!")
        return False

    print(f"\nNote to delete: {note['title']}")
    print(f"Content preview: {note['content'][:100]}...")

    confirm = input("\nAre you sure you want to delete this note? (yes/no): ").lower()

    if confirm == 'yes':
        success = note_manager.delete_note(note_id)
        if success:
            print("Note deleted successfully!")
            return True
        else:
            print("Failed to delete note!")
            return False
    else:
        print("Deletion cancelled.")
        return False

# List all notes
list_all_notes()



ALL NOTES
1. Khuzi
   ID: 0034ecda
   Category: my
   Tags: mine
   Updated: 2025-08-10 22:41:02
   Words: 5



[{'id': '0034ecda',
  'title': 'Khuzi',
  'content': 'Khuzi WUzi loves yiu\njhuzi',
  'category': 'my',
  'tags': ['mine'],
  'created_at': '2025-08-10 22:41:02',
  'updated_at': '2025-08-10 22:41:02',
  'word_count': 5}]

In [6]:

def show_statistics():
    """Display detailed statistics about notes"""
    stats = note_manager.get_stats()
    categories = note_manager.get_categories()
    tags = note_manager.get_tags()

    print("NOTE STATISTICS")
    print("=" * 50)

    print(f"Total Notes: {stats['total_notes']}")
    print(f"Total Words: {stats['total_words']}")
    print(f"Categories: {stats['categories']}")
    print(f"Tags: {stats['tags']}")

    if stats['total_notes'] > 0:
        avg_words = stats['total_words'] / stats['total_notes']
        print(f"Average Words per Note: {avg_words:.1f}")

    print("\nCATEGORIES:")
    print("-" * 20)
    for category, count in sorted(categories.items(), key=lambda x: x[1], reverse=True):
        print(f"{category}: {count} notes")

    if tags:
        print("\nTOPS TAGS:")
        print("-" * 20)
        sorted_tags = sorted(tags.items(), key=lambda x: x[1], reverse=True)[:10]
        for tag, count in sorted_tags:
            print(f"{tag}: {count} notes")

    print()

def browse_by_category():
    """Browse notes by category"""
    categories = note_manager.get_categories()

    if not categories:
        print("No categories found!")
        return

    print("BROWSE BY CATEGORY")
    print("=" * 50)

    print("Available categories:")
    cat_list = list(categories.keys())
    for i, category in enumerate(cat_list, 1):
        print(f"{i}. {category} ({categories[category]} notes)")

    try:
        choice = int(input(f"\nSelect category (1-{len(cat_list)}): "))
        if 1 <= choice <= len(cat_list):
            selected_category = cat_list[choice - 1]

            # Get notes in this category
            category_notes = [note for note in note_manager.notes.values()
                            if note['category'] == selected_category]
            category_notes.sort(key=lambda x: x['updated_at'], reverse=True)

            print(f"\nNotes in '{selected_category}' category:")
            print("=" * 40)

            for i, note in enumerate(category_notes, 1):
                print(f"{i}. {note['title']}")
                print(f"   ID: {note['id']}")
                print(f"   Updated: {note['updated_at']}")
                print(f"   Words: {note['word_count']}")
                print()
        else:
            print("Invalid selection!")
    except ValueError:
        print("Please enter a valid number!")

def browse_by_tags():
    """Browse notes by tags"""
    tags = note_manager.get_tags()

    if not tags:
        print("No tags found!")
        return

    print("BROWSE BY TAGS")
    print("=" * 50)

    print("Available tags:")
    tag_list = sorted(tags.items(), key=lambda x: x[1], reverse=True)[:20]

    for i, (tag, count) in enumerate(tag_list, 1):
        print(f"{i}. {tag} ({count} notes)")

    try:
        choice = int(input(f"\nSelect tag (1-{len(tag_list)}): "))
        if 1 <= choice <= len(tag_list):
            selected_tag = tag_list[choice - 1][0]

            # Get notes with this tag
            tagged_notes = [note for note in note_manager.notes.values()
                          if selected_tag in note['tags']]
            tagged_notes.sort(key=lambda x: x['updated_at'], reverse=True)

            print(f"\nNotes tagged with '{selected_tag}':")
            print("=" * 40)

            for i, note in enumerate(tagged_notes, 1):
                print(f"{i}. {note['title']}")
                print(f"   ID: {note['id']}")
                print(f"   Category: {note['category']}")
                print(f"   Updated: {note['updated_at']}")
                print()
        else:
            print("Invalid selection!")
    except ValueError:
        print("Please enter a valid number!")

# Show statistics
show_statistics()


NOTE STATISTICS
Total Notes: 1
Total Words: 5
Categories: 1
Tags: 1
Average Words per Note: 5.0

CATEGORIES:
--------------------
my: 1 notes

TOPS TAGS:
--------------------
mine: 1 notes



In [7]:

def main_menu():
    """Main application menu with all features"""
    while True:
        print("\nNOTE-TAKING APP MENU")
        print("=" * 50)
        print("1. Create new note")
        print("2. Search notes")
        print("3. List all notes")
        print("4. View specific note")
        print("5. Edit note")
        print("6. Delete note")
        print("7. Browse by category")
        print("8. Browse by tags")
        print("9. Show statistics")
        print("10. Export notes (JSON)")
        print("0. Exit")

        try:
            choice = int(input("\nEnter your choice (0-10): "))

            if choice == 0:
                print("Thank you for using the Note-Taking App!")
                break
            elif choice == 1:
                create_note_interface()
            elif choice == 2:
                search_notes_interface()
            elif choice == 3:
                list_all_notes()
            elif choice == 4:
                note_id = input("Enter note ID: ").strip()
                display_note(note_id)
            elif choice == 5:
                edit_note_interface()
            elif choice == 6:
                delete_note_interface()
            elif choice == 7:
                browse_by_category()
            elif choice == 8:
                browse_by_tags()
            elif choice == 9:
                show_statistics()
            elif choice == 10:
                export_notes()
            else:
                print("Invalid choice! Please select 0-10.")

        except ValueError:
            print("Please enter a valid number!")

        input("\nPress Enter to continue...")

def export_notes():
    """Export all notes to JSON format"""
    if not note_manager.notes:
        print("No notes to export!")
        return

    export_data = {
        'notes': list(note_manager.notes.values()),
        'exported_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'total_notes': len(note_manager.notes)
    }

    filename = f"notes_export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

    print("EXPORT NOTES")
    print("=" * 50)
    print("Note: In Colab, this will display the JSON content.")
    print("Copy and save it to a file for backup.")
    print()

    json_content = json.dumps(export_data, indent=2, ensure_ascii=False)
    print(json_content)

    print(f"\nExported {len(note_manager.notes)} notes successfully!")

# Start the main application
print("Starting Note-Taking App...")
main_menu()

Starting Note-Taking App...

NOTE-TAKING APP MENU
1. Create new note
2. Search notes
3. List all notes
4. View specific note
5. Edit note
6. Delete note
7. Browse by category
8. Browse by tags
9. Show statistics
10. Export notes (JSON)
0. Exit

Enter your choice (0-10): 0
Thank you for using the Note-Taking App!
